In [10]:
import pandas as pd

In [14]:
orig_data_df = pd.read_csv('../resources/data/nytimes-articles-to-extract-sources.csv', index_col=0)

In [13]:
ls ../resources/data/

academic-datasets/
annotated-source-training-df.csv
articles3.csv
fasttext-data/
news-article-flatlist/
news-categories-training-df.csv
nyt_corpus/
nytimes-articles-to-extract-sources.csv
our-annotated-data-full.csv.gz
our-annotated-source-training-df.tsv
output/
polnear-training-data-stage-1.csv
polnear-training-data.csv


In [3]:
db_pathway = '../../edit-pathways/data/diffengine-diffs/spark-output'

In [16]:
ls $db_pathway

ap-matched-sentences.db.gz             independent-matched-sentences.db.gz
bbc-2-matched-sentences.db.gz          lapresse-matched-sentences.db.gz
calgaryherald-matched-sentences.db.gz  nationalpost-matched-sentences.db.gz
canadaland-matched-sentences.db.gz     nyt-matched-sentences.db.gz
cbc-matched-sentences.db.gz            reuters-matched-sentences.db.gz
dailymail-matched-sentences.db.gz      telegraph-matched-sentences.db.gz
fox-matched-sentences.db.gz            torontostar-matched-sentences.db.gz
globemail-matched-sentences.db.gz      wp-matched-sentences.db.gz
guardian-matched-sentences.db.gz


In [35]:
! gunzip $db_pathway/bbc-2-matched-sentences.db.gz

In [1]:
import sqlite3
import pandas as pd 
import os 

In [36]:
db_path = os.path.join(db_pathway, 'nyt-matched-sentences.db')
db_path = os.path.join(db_pathway, 'bbc-2-matched-sentences.db')

In [37]:
conn = sqlite3.connect(db_path)
doc_level_stats_df = pd.read_sql('select * from doc_level_stats', con=conn)

In [39]:
doc_level_stats_df

,entry_id,version_x,version_y,mean x dist,mean y dist,num_unchanged,num_edited_x,num_edited_y,num_deleted,num_added,num_merged,num_split,num_sentences_x,num_sentences_y,num_refactors,mean_dist_overall
0,218742,0,1,0.212504,0.209759,1,3,3,5,8,0,0,9,12,0,0.211132
1,218742,1,2,0.222330,0.222819,3,8,8,3,5,0,0,14,16,3,0.222575
2,218742,2,3,0.032382,0.054402,13,3,4,0,2,0,1,16,19,0,0.043392
3,218742,3,4,0.000000,0.000000,19,0,0,0,0,0,0,19,19,0,0.000000
4,218742,4,5,0.063167,0.081335,13,6,7,0,0,0,1,19,20,0,0.072251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506896,2023662,0,1,0.025225,0.025225,0,4,4,0,2,0,0,4,6,0,0.025225
506897,2023662,1,2,0.006874,0.006874,8,1,1,0,0,0,0,9,9,0,0.006874
506898,2023672,0,1,0.103980,0.104316,7,8,8,1,3,0,0,16,18,2,0.104148
506899,2023684,0,1,0.000000,0.000000,2,0,0,0,0,0,0,2,2,0,0.000000


In [41]:
num_versions = doc_level_stats_df['entry_id'].value_counts().to_frame('num_versions')
candidate_articles = (
    doc_level_stats_df
    .merge(num_versions, left_on='entry_id', right_index=True)
#     .loc[lambda df: (df['version_x'] / df['num_versions']) < .2]  ## articles early in their lifecycle
#     .loc[lambda df: df['num_sentences_x'] < 20]                   ## short articles
    .loc[lambda df: df['num_added'] / df['num_sentences_x'] > .2] ## articles that grow between iterations
    .loc[lambda df: df['num_deleted'] / df['num_sentences_x'] < .05]
)

In [42]:
candidate_articles

,entry_id,version_x,version_y,mean x dist,mean y dist,num_unchanged,num_edited_x,num_edited_y,num_deleted,num_added,num_merged,num_split,num_sentences_x,num_sentences_y,num_refactors,mean_dist_overall,num_versions
13,218751,0,1,1.136929e-01,1.136929e-01,4,3,3,0,2,0,0,7,9,0,1.136929e-01,2
17,218753,1,2,1.250466e-01,1.695410e-01,3,3,4,0,2,0,1,6,9,0,1.472938e-01,2
19,218759,0,1,5.478539e-02,5.430891e-02,0,4,4,0,5,0,0,4,9,0,5.454715e-02,2
34,218779,0,1,1.018364e-02,1.018364e-02,17,1,1,0,4,0,0,18,22,0,1.018364e-02,1
35,218780,0,1,6.391144e-02,6.391144e-02,4,4,4,0,10,0,0,8,18,0,6.391144e-02,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506820,2023422,7,8,4.602415e-02,3.876708e-02,47,8,7,1,14,3,2,56,68,4,4.239562e-02,9
506824,2023444,0,1,5.807032e-07,5.807032e-07,0,5,5,0,17,0,0,5,22,0,5.807032e-07,7
506838,2023477,0,1,1.255753e-02,1.255753e-02,10,3,3,0,3,0,0,13,16,0,1.255753e-02,2
506845,2023486,2,3,5.393440e-02,5.425740e-02,1,7,7,0,3,0,0,8,11,0,5.409590e-02,4


In [43]:
keys = (
    candidate_articles
     # unstack and get all version information
     .set_index('entry_id')[['version_x', 'version_y']]
     .unstack()
     .reset_index()
     .rename(columns={0: 'version'})
     [['entry_id', 'version']]
     .sort_values(['entry_id', 'version'])
     # generate keys
     .pipe(lambda df: df.apply(lambda x: '%s-%s' % (
                int(x['entry_id']),
                int(x['version'])
            )
     , axis=1)
    )
).tolist()
keys = list(map(lambda x: "'%s'" % x, keys))
keys = ', '.join(keys)

In [44]:
candidate_article_df = pd.read_sql(
'''WITH c1 as ( 
        SELECT *, 
        CAST(entry_id AS INT) || '-' || CAST(version AS INT) as key
        FROM split_sentences
        )
        SELECT entry_id, version, sent_idx, sentence
        FROM c1
        WHERE key in (%s)''' % keys
, con=conn)

In [45]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from string import punctuation
import fasttext
from sklearn.metrics import f1_score, precision_score
import numpy as np 

def replace_punct(s):
    for p in punctuation:
        s = s.replace(p, ' ')
#     for w in ENGLISH_STOP_WORDS:
#         s = s.replace(' %s ' % w, ' ')
    s = ' '.join(s.split())
    return s

model = fasttext.load_model('../resources/data/fasttext-data/model_news-classification.bin')

grouped_articles = (candidate_article_df
                    .groupby(['entry_id', 'version'])['sentence'].aggregate(list).apply(lambda x: ' '.join(x)))

X_test = (
    grouped_articles
     .fillna('')
     .str.replace('\n', ' ')
     .str.lower()
     .apply(replace_punct)
     .str.strip()
)

In [46]:
y_pred = X_test.apply(model.predict)
y_pred_final = (y_pred
 .to_frame('output')
 .assign(
     y_pred=lambda df: 
         df['output']
             .str.get(0)
             .str.get(0)
             .str.replace('__label__', '')
             .str.strip()
        )
 .assign(y_prob=lambda df: df['output'].str.get(1).apply(lambda x: x[0] if len(x) > 0 else np.nan))
 [['y_pred', 'y_prob']]
)

In [47]:
categories = (y_pred_final
 .reset_index()
 .assign(index=lambda df: df['entry_id'].astype(str) + '-' + df['version'].astype(str))
 [['index', 'y_pred']]
)

In [48]:
candidate_article_df = candidate_article_df.sort_values(['entry_id', 'version', 'sent_idx'])

In [49]:
groupby_docs = candidate_article_df.groupby(['entry_id', 'version']).aggregate(list)
groupby_docs = groupby_docs['sentence']

In [50]:
groupby_docs = (
    groupby_docs.reset_index()
   .assign(doc_idx=lambda df: df['entry_id'].astype(int).astype(str) + '-' + df['version'].astype(int).astype(str))
 [['doc_idx', 'sentence']]
   .merge(categories, left_on='doc_idx', right_on='index')
   .drop('index', axis=1)
)

In [51]:
to_output = groupby_docs.to_dict(orient='records')

In [52]:
import jsonlines

In [53]:
with open('../tasks/quote_detection_and_attribution/data_to_score_bbc.jsonl', 'w') as f:
    jsonlines.Writer(f).write_all(to_output)

In [137]:
with open('../tasks/quote_detection_and_attribution/data_to_score_with_categories.jsonl', 'w') as f:
    jsonlines.Writer(f).write_all(
        groupby_docs
        .loc[lambda df: df['y_pred'] != 'Other']
        .to_dict(orient='records')
    )

In [139]:
categories.to_csv('../tasks/quote_detection_and_attribution/categories.csv')